## Columbia University
### ECBM E4040 Neural Networks and Deep Learning. Fall 2021.

# ECBM E4040 - Assignment 2 - Task 4: Data Augmentation & Transfer Learning 

Important factors in neural network training include the size and quality of the training set. Since it is often not possible to get a clean and large enough dataset for training, one way to improve the network's robustness and generalization ability is to create 'fake' data by injecting random noise or to perform transformations on the available data. A technique which implements this strategy is called __data augmentation__ and has shown to be very effective.

One thing to remember when you augment your data is to never change the correct label of a sample. For example, for hand-written digit dataset, flipping a letter 'b' ends up looking like a letter 'd', but you must keep the label for 'b'. So please choose the best augmentation methods for your dataset.

In the last part of this task, we introduce __transfer learning__ in TensorFlow to you by showing a demo. Test time augmentation (TTA) as additional content in data augmentation is also introduced based on your experiences on previous parts.

**Note:** you will need to download the data for task 4, located in the liondrive [here](https://drive.google.com/drive/folders/1SLLr3sEiZCmldrhX46f9_Nhkd-7PmV_t). You should have two .csv files in the folder '__data__': '__sign_mnist_test.csv__' and '__sign_mnist_train.csv__'.

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

# Import modules
import numpy as np
import os

try:
    import pandas as pd
    
except ModuleNotFoundError:
    os.system('pip install pandas')
    import pandas as pd
    
import matplotlib.pyplot as plt
import tensorflow as tf

## Load Data

For this task we will be using the Sign Language MNIST dataset. Examples are comprised of 28 x 28 greyscale images representing sign language letters. Note that while most sign letter are static symbols, the signs for J (9) and Z (25) include motions and are thus not included in this dataset.


![image](https://s-media-cache-ak0.pinimg.com/564x/f9/68/ee/f968eece212b201510fd617c9cb70269.jpg)

In [ ]:
# Load the Sign Language MNIST data.
img_cols = 28
img_rows = 28

#Import data from csv using pandas DataFrame
X_train = pd.read_csv("./data/sign_mnist_train.csv")
X_test = pd.read_csv("./data/sign_mnist_test.csv")

#Convert X_train,X_test from DataFrame to an array
X_train = X_train.values
X_test = X_test.values

#First column of X_train is y_train, first column of X_test is y_test
y_train = X_train[:,0]
y_test = X_test[:,0]

#Remove first column from X_train, X_test
X_train = X_train[:,1:]
X_test = X_test[:,1:]

# Data organizations:
# Train data: 27455 samples from original train set: 1~27455
# Validation data: 1000 samples from original train set: 26455~27455
# Test data: 7172 samples
# We've vectorized the 

X_train = X_train.reshape(27455, -1)
X_test = X_test.reshape(7172,-1)

num_train = 26455
num_validation = 1000
num_dev = 128

# The development set is used for augmentation practices.
mask = np.random.choice(num_train, num_dev, replace=False)
X_dev = X_train[mask]
y_dev = y_train[mask]
# Seperate Training set into a training set and a validation set
X_val = X_train[num_train:]
y_val = y_train[num_train:]
X_train = X_train[:num_train]
y_train = y_train[:num_train]

print(X_train.shape, X_val.shape, X_test.shape, X_dev.shape)

print("Number of classes: {}".format(len(set(y_train))))

## Part 1: Visualization

### Visualize some original images

Here we use Pyplot to draw any 16 samples from the __development set__ in a 4-by-4 grid.

__Note__: Since we have vectorized our data, we need to reshape it into 28 x 28 greyscale images.

In [ ]:
# Visualization of the original data.

fig = plt.figure(figsize=(10, 10))

for i in range(16):
    ax = fig.add_subplot(4, 4, i+1)
    ax.imshow(X_dev[i, :].reshape(28, 28), 'gray')
    ax.axis('off')

## Part 2: Automatic Batch Generator

We want you to create an automatic image generator that does several kinds of data augmentations, and produces a batch of data consisting of random samples every time you call it. 

<span style="color:red">__TODO__:</span> Finish the functions of class __ImageGenerator__ in **utils/image_generator.py**. The code is fully commented with instructions.

__Hint__: The python keywords __yield__ and __next__ can help you do some tricks.

In [ ]:
from utils.image_generator import ImageGenerator

Create an ImageGenerator object using the __development set__, and use __show__ function to plot the top 16 original images.

__Note__: We need to reshape your data as the demanding input format of the class __ImageGenerator__.

In [ ]:
print(X_dev.shape)
print(X_dev.reshape(-1,1,28,28).transpose(0,2,3,1).shape)

In [ ]:
gen = ImageGenerator(X_dev.reshape(-1,1,28,28).transpose(0,2,3,1), y_dev)
gen.show(X_dev)

### Increase Brightness (demo)

Increase the brightness of the original __development set__, and plot 16 images with noise added.(These images may not be the same as the above 16 images according to your choice of parameters.)

In [ ]:
bright = gen.brightness(1.5)
gen.show(bright)

### Horizontal and Verical Flip (demo)

Flip the original __development set__ as you like (horizontal, vertical, or both), and plot the top 16 images.

In [ ]:
flipped = gen.flip('v')
gen.show(flipped)

### Translation

<span style="color:red">__TODO:__</span> Implement the funciton **translate()** in **utils/image_generator.py**. Shift the original __development set__ by several pixels in both directions, and plot the top 16 images.

In [ ]:
### TODO: Your code here ###

### Rotation

<span style="color:red">__TODO:__</span> Implement the function **rotate()** in **utils.image_generator.py**. Rotate the original __development set__ by several degrees and plot the top 16 images.### TODO: Your code here ###

In [ ]:
### TODO: Your code here ###

### Add Noise

<span style="color:red">__TODO:__</span> Implement the function **add_noise()** in **utils.image_generator.py**. Inject random noise into the original __development set__, and plot 16 images with noise added. (These images may not be the same as the above 16 images according to your choice of parameters.)

In [ ]:
### TODO: Your code here ###

## Part 3: Data Augmentation + LeNet

<span style="color:red">__TODO__:</span> Now you have your own data generator. You have been provided a simplified LeNet model in __utils/neuralnets/cnn/model_LeNet.py__. In __utils/neuralnets/cnn/my_LeNet_trainer.py__ you will find two TODOs. 

The first TODO asks you to **prepare batches of augmented training data** using the ImageGenerator you completed in the previous section. When generating augmented data, it is important to think carefully about the changes you are making to the data and the effect those changes may have on their classification. For example, the sign for the letter G is as follows:

![im](https://media.istockphoto.com/vectors/is-the-seventh-letter-of-the-alphabet-in-sign-language-vector-image-vector-id1209943258?k=6&m=1209943258&s=612x612&w=0&h=46JNhmLyb9qHjNiI7TGC9xKo7icHf7GEW6NA2Z25c6A=)

Similarly, the sign for Q is 

![im2](https://data.formsbank.com/pdf_docs_html/35/354/35435/page_1_thumb_big.png).

<span style="color:red">__TODO__:</span> Before completing the TODOs in __utils/neuralnets/cnn/my_LeNet_trainer.py__, explain which augmentations, or combination of augmentations (brightness, horizontal flip, vertical flip, translation, rotation, noise, etc.) could create issues for the classifier in the context of the signs for G and Q. Are there any other augmentations of signs that could pose problems?

<span style="color:red">__Your answer here__:</span>

<span style="color:red">__TODO__:</span> Propose a series of augmentations (at least 4) from the functions we implemented to generate an augmented training set. Be sure to avoid the issues discussed above and include argument parameters for the augmentation functions.

Ex:

1. flip('v')
2. brightness(1.5)
3. etc.

<span style="color:red">__Your answer here__:</span>

<span style="color:red">__TODO__:</span> Complete the function **batch_train_data()** in **utils/neuralnets/cnn/my_LeNet_trainer.py** with your proposed data augmnetations.

<span style="color:red">__TODO__:</span> Complete the function **train_epoch()** in **utils/neuralnets/cnn/my_LeNet_trainer.py**.

__Install tqdm__: tqdm is a fast, extensible progress meter.

In [ ]:
!pip install tqdm

In [ ]:
X_t = X_train.reshape(-1,1,28,28).transpose(0,2,3,1)
X_v = X_val.reshape(-1,1,28,28).transpose(0,2,3,1)

print(X_t.shape)
print(X_v.shape)

In [ ]:
from utils.neuralnets.cnn.my_LeNet_trainer import MyLeNet_trainer

<span style="color:red">__TODO__:</span> train network using run method in MyLeNet_trainer

In [ ]:
# YOUR CODE HERE

In [ ]:
# TODO: add describe training process

## Part 4 (demos): Transfer Learning and Test Time Augmentation (TTA)

This is the last part of assignment 2 before we go to the kaggle competition. In this part, we will provide you with examples of transfer learning, as well as the last context of data augmentation: test time augmentation. We believe they are useful tools for task 5.

__Note__: The network in demos is not guaranteed to be well-trained. __No points are set in this part__. Feel free to edit the scripts and tune the parameters by yourselves.

### Transfer Learning

The intuition behind transfer learning is that if a model is trained on a large and general enough dataset, this model will effectively serve as a generic model of the visual world. For a somewhat different problem, you can then take advantage of these learned feature maps without having to start from scratch and training a new large model on a large dataset.

A pre-trained model is a saved network that was previously trained on a large dataset, typically on a large-scale image-classification task. Generally, we have two ways to customize a pre-trained model:

1. **Feature Extraction**: Use the representations learned by a previous network to extract meaningful features from new samples. You simply add a new classifier, which will be trained from scratch, on top of the pretrained model so that you can repurpose the feature maps learned previously for our dataset. However, the final, classification part of the pretrained model is specific to original classification task, and subsequently specific to the set of classes on which the model was trained. That means you do not need to (re)train the entire model. You "freeze" the base convolution network, and only train the newly added classifier layers. 

2. **Fine-Tuning**: Unfreezing a few of the top layers of a frozen model base and jointly training both the newly-added classifier layers and the last layers of the base model. This allows us to "fine tune" the higher-order feature representations in the base model in order to make them more relevant for the specific task.

This example uses the base model from __MobileNet__ for a transfer learning of a 10-class classification task on CIFAR-10 dataset. The whole pipeline will include:

1. Load data
2. Build an input pipeline, in this case using Keras ImageDataGenerator
3. Compose our model
4. Load in our pretrained base model (and pretrained weights)
5. Stack our classification layers on top
6. Train our model
7. Evaluate model

Other references: https://towardsdatascience.com/keras-transfer-learning-for-beginners-6c9b8b7143e

In [ ]:
import pandas as pd
import os
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from utils.cifar_utils import load_data

In [ ]:
# Load the raw CIFAR-10 data.
X_train, y_train = load_data(mode='train')

# Data organizations:
# Train data: 49000 samples from original train set: 1~49000
# Validation data: 1000 samples from original train set: 49000~50000
num_training = 49000
num_validation = 1000

X_val = X_train[-num_validation:, :]
y_val = y_train[-num_validation:]

X_train = X_train[:num_training, :]
y_train = y_train[:num_training]

# Preprocessing: subtract the mean value across every dimension for training data, and reshape it to be RGB size
mean_image = np.mean(X_train, axis=0)
X_train = X_train.astype(np.float32) - mean_image.astype(np.float32)
X_val = X_val.astype(np.float32) - mean_image

X_train = X_train.reshape(-1,3,32,32).transpose(0,2,3,1) / 255
X_val = X_val.reshape(-1,3,32,32).transpose(0,2,3,1) / 255

print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)

y_train_dummy = tf.keras.utils.to_categorical(y_train)
y_val_dummy = tf.keras.utils.to_categorical(y_val)
print('Train labels shape (one-hot): ', y_train_dummy.shape)
print('Validation labels shape (one-hot): ', y_val_dummy.shape)

In [ ]:
#imports the mobilenet model and discards the last 1000 neuron layer.
base_model=MobileNet(weights='imagenet',include_top=False) 

x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x=Dense(1024,activation='relu')(x) #dense layer 2
x=Dense(512,activation='relu')(x) #dense layer 3
preds=Dense(10,activation='softmax')(x) #final layer with softmax activation

In [ ]:
model=Model(inputs=base_model.input,outputs=preds)
#specify the inputs
#specify the outputs
#now a model has been created based on our architecture

In [ ]:
for layer in model.layers[:20]:
    layer.trainable=False
for layer in model.layers[20:]:
    layer.trainable=True

In [ ]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
# Adam optimizer
# loss function will be categorical cross entropy
# evaluation metric will be accuracy
model.summary()

In [ ]:
bs = 98 #batch size
epc = 25 #number of epoches
train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input) #included in our dependencies
train_generator = train_datagen.flow(X_train, y_train_dummy, batch_size=bs)
step_size_train=train_generator.n//train_generator.batch_size

In [ ]:
# train the model
history = model.fit_generator(generator = train_generator,
                   steps_per_epoch = step_size_train,
                   epochs = epc,
                   validation_data=(X_val, y_val_dummy))

In [ ]:
print(f'Baseline accuracy: {model.evaluate(X_val, y_val_dummy)[1]}')

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')

plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

### Test Time Augmentation (TTA)

Key references: 

https://towardsdatascience.com/test-time-augmentation-tta-and-how-to-perform-it-with-keras-4ac19b67fb4d

https://machinelearningmastery.com/how-to-use-test-time-augmentation-to-improve-model-performance-for-image-classification/


Data Augmentation is the process of randomly applying some operations (rotation, zoom, shift, flips,…) to the input data. By this mean, the model can learn more general features about the classes it has to recognize.

However, there also exists some ways to improve the results of the model by changing the way we test it. That is Test Time Augmentation (TTA).

TTA is now a commonly used technique in kaggle competition on classification. Similar to what data augmentation is doing to the training set, TTA is to perform similar data modifications to the test images. Thus, instead of showing the regular, “clean” images, only once to the trained model, we will show it the augmented images several times. The final guess of each corresponding image will base on the average of the prediction results.

The reason why we refer to TTA is that, by averaging our predictions, on randomly modified images, we are also averaging the errors. The error can be big in a single vector, leading to a wrong answer, but when averaged, only the correct answer stand out. TTA is particularly useful for test images that the model is pretty unsure. The following example will show you how to apply TTA with Keras.


In [ ]:
# call an ImageDataGenerator similar to training set for test set.
test_datagen = ImageDataGenerator(
        shear_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True,
        rotation_range=10.,
        fill_mode='reflect', 
        width_shift_range = 0.1, 
        height_shift_range = 0.1)

In [ ]:
from tqdm import tqdm

# test time augmentation, we set TTA for 10 times averaging.
tta_steps = 10
predictions = []

for i in tqdm(range(tta_steps)):
    preds = model.predict(test_datagen.flow(X_val, batch_size=bs, shuffle=False), steps = len(X_val)/bs)
    predictions.append(preds)

In [ ]:
# print final prediction results
final_pred = np.mean(predictions, axis=0)
print(f'Accuracy with TTA: {np.mean(np.equal(np.argmax(y_val_dummy, axis=-1), np.argmax(final_pred, axis=-1)))}')